In [154]:
import numpy as np
import random
import itertools
import math
import tqdm
from tqdm import trange

In [155]:
# function to distingish whether notes in given timestamp and chord are within or outside the chord (0=outside,1=within)
import json
with open('../modules/json_files/keychorddict.json') as json_file:
    chord_notes = json.load(json_file)
  
    def note_2_class(chord,notes_at_t,chord_notes=chord_notes):
        note_in_chord=chord_notes[chord]['idx']
        return [int(note in note_in_chord) for note in notes_at_t]

In [212]:
### I threat the total length of time as the length of the observered list

class HMM:
    def __init__(self,no_of_state,no_of_value,states,values):
        #randomize all matrix, each row should sum to 1
        #self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        

        #Modification: 2 parameter for all chord  [note inside chord & note outside chord]
        self.emssion_matrix=np.random.rand(no_of_value)
        self.emssion_matrix=self.emssion_matrix/self.emssion_matrix.sum()
        self.emssion_matrix=np.array([self.emssion_matrix,]*len(states)) # assume same probability to emit "note within chord"  for all chords
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        
        self.states=states
        self.values=values
        self.observered=None
        
        #self.prev_initial_matrix=self.initial_matrix
        #self.prev_emssion_matrix=self.emssion_matrix
        #self.prev_transition_matrix=self.transition_matrix
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emission_matrix\n',self.emssion_matrix)
        
    def likelihood(self,state,ob_t):
        
        #Modification: convert observation[t] from notes to 2_classes
        chord=self.states[state]  #numeric to chord name
        ob_t=note_2_class(chord,ob_t) #2 class
        
        prob=1
        for x in ob_t:
            prob*=(self.emssion_matrix[state][x])
        return prob
 
    def forward(self,t,j,ob=None,mode=False):
        if ob is None:
            ob=self.observered
        if t==0:
            if mode==True:
                return self.initial_matrix[j]*self.likelihood(j,ob[t]),0
            else:
                return self.initial_matrix[j]*self.likelihood(j,ob[t])
        else:          
            if mode==True:
                result=np.array([self.forward(t-1,i,ob,mode)[0]*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.likelihood(j,ob[t])
                return np.max(result),np.argmax(result)
            else:
                return sum([self.forward(t-1,i,ob,mode)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.likelihood(j,ob[t])

    def backward(self,t,i,ob=None,mode=False):
        if ob is None:
            ob=self.observered
        if t==len(ob)-1:
            return 1
        else:
            if mode==True:
                return max([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob,mode) for j in range(len(self.states))])
            else:
                return sum([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob,mode) for j in range(len(self.states))])
           
    def probit_at_i(self,t,i,ob=None):#Gamma γt(i) = P(qt = i|O,λ)      
        if ob is None:
            ob=self.observered
        numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
        denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(len(self.states))]) #prob of passing through  ALL_state at time t
        return numerator/denominator
    
    def probit_transit_i_j(self,t,i,j,ob=None):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
        if ob is None:
            ob=self.observered
        numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
        denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.likelihood(n,ob[t+1])*self.backward(t+1,n,ob) for n in range(len(self.states))]) for m in range(len(self.states))]) #prob of ALL transition combination at time t
        return (numerator/denominator)
    def predict(self,ob):
        T1=np.empty((len(self.states),len(ob)),'d')
        T2=np.empty((len(self.states),len(ob)),'B')
        for idx in range(len(self.states)):
            T1[idx,0]=self.forward(0,idx,ob,True)[0]
        T2[:,0]=0
        
        for i in range(1,len(ob)):
            for idx in range(len(self.states)):
                T1[idx,i],T2[idx,i]=self.forward(i,idx,ob,True)
        x = np.empty(len(ob), 'B')
        x[-1] = np.argmax(T1[:, len(ob) - 1])
        for i in reversed(range(1, len(ob))):
            x[i - 1] = T2[x[i], i]
        return x
        
        
    def train(self,obs=None,epochs=2):
        #O:observed values
        #λ:model parameters
         
        if obs is None:
            obs=self.observered
        
        for epoch in range(epochs):
            for ob in obs:
                
                #initial matrix
                for i in range(len(self.states)):
                    self.initial_matrix[i]=self.probit_at_i(0,i,ob)
                #transition matrix
                for i, j in itertools.product(range(len(self.states)),range(len(self.states))):
                    self.transition_matrix[i][j]=sum([self.probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([self.probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                #emission matrix
                for j, k in itertools.product(range(len(self.states)),range(len(self.values))):   
                    total=0
                    
                    #Modification: convert notes to 2 classes (outside or inside given chord)
                    chord=self.states[j]  #numeric to chord name
                    ob_2_class=[note_2_class(chord,ob_t) for ob_t in ob] #2 class

                    for t in range(len(ob)):
                        if k in ob_2_class[t]:
                            #Modification: multiple by how many times do k appear at timestamp t
                            total+=self.probit_at_i(t,j,ob)*ob_2_class[t].count(k)  
                            
                    #Modification: multiple by len(ob[t]), which is the total length of notes at timestamp t                    
                    self.emssion_matrix[j][k]=total/sum([self.probit_at_i(t,j,ob)*len(ob[t]) for t in range(len(ob))])  
                    
                    #smoothing
                    if self.emssion_matrix[j][k]==0:
                        self.emssion_matrix[j][k]=0.0000000000000000001

In [213]:
import json
with open('../data/training_data.json') as json_file:
    data = json.load(json_file)

In [214]:
h_states=["GbMajorI","GbMajorI7","GbMajorbII","GbMajorII","GbMajorII7",
         "GbMajorIII","GbMajorIII7","GbMajorIV","GbMajorIV7","GbMajorV",
         "GbMajorV7","GbMajorbVI","GbMajorGerVI","GbMajorFreVI","GbMajorItaVI",
         "GbMajorVI","GbMajorVI7","GbMajorVII","GbMajorVII7","GbMajorDimVII7",
]

In [215]:
#Filter for Gb Major
select_idx=[]
for idx,chord in enumerate(data['Etude_in_Gb_Major.mxl']['chord_seq']):
    if chord.startswith('GbM'):
        select_idx.append(idx)

In [216]:
test=HMM(len(h_states),2,h_states,["outside chord","inside chord"])

In [222]:
training=[]
training_label=[]
for idx in select_idx[10:12]:
    training.append(data['Etude_in_Gb_Major.mxl']['note_seq'][idx])
    training_label.append(data['Etude_in_Gb_Major.mxl']['chord_seq'][idx])

In [223]:
key_mapping={
    'C':0,
    'D':2,
    'E':4,
    'F':5,
    'G':7,
    'A':9,
    'B':11
}
def key2num(key):  
  key=key.upper()
  num=key_mapping[key[0]]
  modifier=len(key)
  if modifier==1:
    return num
  elif key[1]=='#':
    return (num+(modifier-1))%12
  elif key[1]=='B' or key[1]=='-':
    return (num-(modifier-1))%12
  elif key[1]=='X':
    return (num+(modifier-1)*2)%12

# key_list to number_list
def keys2num(keys):
    return [key2num(key) for key in keys]

In [224]:
training=[keys2num(segment)for segment in training]

In [225]:
test.train([training],2)

KeyboardInterrupt: 

In [226]:
prediction=test.predict(training)
prediction=[h_states[idx] for idx in prediction]
(prediction,training_label)

(['GbMajorbII',
  'GbMajorVII',
  'GbMajorI',
  'GbMajorIII',
  'GbMajorDimVII7',
  'GbMajorI7'],
 ['GbM_I', 'GbM_IV', 'GbM_I', 'GbM_II', 'GbM_III', 'GbM_I'])

In [227]:
test.debug()

initial_matrix
 [4.02403307e-10 2.80730964e-06 7.42098418e-01 1.81372370e-06
 1.20485349e-53 1.79510908e-10 8.13600803e-31 4.02924732e-83
 2.01816276e-23 2.17526258e-56 5.25853434e-52 1.40486551e-07
 4.28156018e-07 1.79821576e-07 1.16351083e-06 6.36447679e-07
 4.97881174e-10 2.82448496e-06 2.24364324e-04 8.30918947e-06]
transition_matrix
 [[4.50987386e-13 9.18989696e-05 5.44626785e-05 4.72620873e-04
  1.09419026e-14 1.54051961e-02 4.30832784e-01 1.74219742e-27
  8.17696170e-23 7.57772271e-02 2.49022871e-01 3.36201893e-06
  3.89528085e-06 6.10688792e-05 2.59509936e-06 6.55696431e-08
  2.98441537e-13 1.09974872e-03 2.72355839e-04 4.06557630e-04]
 [8.77927073e-14 8.96933305e-05 6.48617775e-06 1.01658936e-04
  5.26621944e-15 3.24684786e-03 8.60614966e-01 1.37284428e-28
  5.27177266e-23 2.59990617e-02 5.43739049e-02 3.15343669e-06
  2.82414173e-06 4.87841218e-07 3.98460445e-06 2.22748505e-07
  1.52061435e-13 5.74574546e-04 9.33574043e-04 1.00919641e-03]
 [5.35315862e-02 1.72925562e-01 1.401